In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-users/users (2).json
/kaggle/input/user-vgg-features/user_features.json


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

2024-06-04 13:11:44.291637: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 13:11:44.291748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 13:11:44.390709: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import json
import pandas as pd

# Load JSON data
with open('/kaggle/input/user-vgg-features/user_features.json', 'r') as file:
    features = json.load(file)


In [4]:
with open('/kaggle/input/dataset-users/users (2).json','r') as file: 
    users = json.load(file)

In [5]:
# Prepare input X and target y
X = []
y = []

for user in users:
    user_id = user['user_id']
    if user_id in features:
        # Add the feature vector to X
        X.append(features[user_id])

        # Extract sub-interests
        interests = user.get('interests', {})
        sub_interests = [sub_interest for category in interests.values() for sub_interest in category]
        y.append(sub_interests)

In [6]:
# Convert X and y to numpy arrays
X = np.array(X)

# Binarize the labels for multi-label classification
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

In [7]:
from collections import defaultdict

# Calculate the number of instances for each class in the target labels
class_instances = defaultdict(int)
for labels in y:
    for i, label in enumerate(labels):
        if label == 1:
            class_instances[i] += 1

In [8]:
threshold = 70

# Identify minor classes with fewer instances
minor_classes = [class_idx for class_idx, count in class_instances.items() if count < threshold]

# Define augmentation parameters
noise_std = 0.1  # Standard deviation of Gaussian noise
min_scale = 0.8  # Minimum scaling factor
max_scale = 1.2  # Maximum scaling factor

In [9]:
def add_gaussian_noise(feature_vector):
    noise = np.random.normal(loc=0, scale=noise_std, size=feature_vector.shape)
    return feature_vector + noise

# Function to perform random scaling on a feature vector
def random_scaling(feature_vector):
    scale_factor = np.random.uniform(min_scale, max_scale)
    scaled_vector = feature_vector * scale_factor
    return scaled_vector



In [10]:
# Define a function for data augmentation
def augment_data(X):
    # Add Gaussian noise
    X_noise = X + np.random.normal(0, 0.1, size=X.shape)
    
    # Random scaling
    scaling_factor = np.random.uniform(0.9, 1.1, size=(X.shape[0], 1))
    X_scaled = X * scaling_factor
    
    # Concatenate original data with augmented data
    X_augmented = np.concatenate([X, X_noise, X_scaled], axis=0)
    
    return X_augmented

In [11]:
# Perform data augmentation for minor classes in the training data
augmented_X = list(X)  # Start with original data
augmented_y = list(y)

for i, x in enumerate(X):
    for j, y_label in enumerate(y[i]):
        if j in minor_classes and y_label == 1:
            # Add Gaussian noise
            augmented_feature_vector_1 = add_gaussian_noise(x)
            augmented_X.append(augmented_feature_vector_1)  # Append to the list
            augmented_y.append(y[i])

            # Random scaling
            augmented_feature_vector_2 = random_scaling(x)
            augmented_X.append(augmented_feature_vector_2)  # Append to the list
            augmented_y.append(y[i])

# Convert augmented data to numpy arrays
augmented_X = np.array(augmented_X)
augmented_y = np.array(augmented_y)


In [12]:
augmented_X.shape

(2992, 1, 512)

In [13]:
augmented_X = np.array(augmented_X).reshape(len(augmented_X), -1)


In [14]:
augmented_X.shape

(2992, 512)

In [15]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score

class F1ScoreCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(X_test)
        y_pred = (y_pred > 0.5).astype(int)
        f1 = f1_score(y_test, y_pred, average='micro')
        print(f' - val_f1_score: {f1:.4f}')

In [16]:
X_train, X_test, y_train, y_test = train_test_split(augmented_X, augmented_y, test_size=0.2, random_state=42)

In [17]:
# Define the neural network model
vgg2 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(y.shape[1], activation='sigmoid') 
])
# Compile the model
vgg2.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
vgg2.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2,validation_data=(X_test, y_test), callbacks=[F1ScoreCallback()])

Epoch 1/50
62/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0427 - loss: 0.5638

I0000 00:00:1717507131.921624     119 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1717507131.937471     119 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0472 - loss: 0.5164

W0000 00:00:1717507134.486198     117 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1717507134.835459     119 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
 - val_f1_score: 0.1263
75/75 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step - accuracy: 0.0476 - loss: 0.5133 - val_accuracy: 0.2037 - val_loss: 0.1073
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1767 - loss: 0.137
 - val_f1_score: 0.2710
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1805 - loss: 0.1353 - val_accuracy: 0.3823 - val_loss: 0.0852
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2774 - loss: 0.107
 - val_f1_score: 0.2915
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2810 - loss: 0.1066 - val_accuracy: 0.4357 - val_loss: 0.0791
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3512 - loss: 0.090
 - val_f1_score: 0.4158
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3537 - loss: 0.0899 - val_accuracy: 0.4858 - val_loss: 0.0664
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4120 - loss: 0.079
 - val_f1_score: 0.4759
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accurac

In [21]:
# Evaluate the model
y_pred = vgg2.predict(X_test)
print(classification_report(y_test, y_pred > 0.5, target_names=mlb.classes_))

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
                                precision    recall  f1-score   support

                   Auto racing       1.00      1.00      1.00        21
                      Business       1.00      0.46      0.63        50
                         Games       1.00      0.50      0.67        14
                   Health care       1.00      0.94      0.97        16
             Physical exercise       0.92      0.94      0.93        35
            alcoholic beverage       1.00      0.94      0.97        16
             american football       1.00      1.00      1.00        10
                 art and music       1.00      0.62      0.76        13
                      baseball       1.00      1.00      1.00         5
                    basketball       1.00      0.57      0.73        14
                        beauty       1.00      1.00      1.00        40
                     beverages       1.00      0.95      0.97        19
                  bodyb

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
vgg3 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(49, activation='sigmoid')
])

vgg3.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
vgg3.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2,validation_data=(X_test, y_test), callbacks=[F1ScoreCallback()])

Epoch 1/50
53/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0284 - loss: 0.9073

W0000 00:00:1717507157.167632     118 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.0264 - loss: 0.8045

W0000 00:00:1717507165.753301     118 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1717507166.176561     119 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 1/19 ━━━━━━━━━━━━━━━━━━━━ 3s 219ms/step

W0000 00:00:1717507166.444099     116 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
 - val_f1_score: 0.0000
75/75 ━━━━━━━━━━━━━━━━━━━━ 16s 129ms/step - accuracy: 0.0263 - loss: 0.8007 - val_accuracy: 0.0100 - val_loss: 0.1644
Epoch 2/50
54/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0178 - loss: 0.2394   

W0000 00:00:1717507166.661559     117 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
 - val_f1_score: 0.0000
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0189 - loss: 0.2335 - val_accuracy: 0.0033 - val_loss: 0.1467
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0306 - loss: 0.186
 - val_f1_score: 0.0000
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0303 - loss: 0.1852 - val_accuracy: 0.0050 - val_loss: 0.1494
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0244 - loss: 0.1745  
 - val_f1_score: 0.0000
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0234 - loss: 0.1744 - val_accuracy: 0.0033 - val_loss: 0.1455
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0352 - loss: 0.170
 - val_f1_score: 0.0000
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0344 - loss: 0.1691 - val_accuracy: 0.0033 - val_loss: 0.1449
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0242 - loss: 0.1597  
 - val_f1_score: 0.0000
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - a

In [23]:
y_pred = vgg3.predict(X_test)
print(classification_report(y_test, y_pred > 0.5, target_names=mlb.classes_))

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
                                precision    recall  f1-score   support

                   Auto racing       0.00      0.00      0.00        21
                      Business       0.00      0.00      0.00        50
                         Games       0.00      0.00      0.00        14
                   Health care       0.00      0.00      0.00        16
             Physical exercise       0.00      0.00      0.00        35
            alcoholic beverage       0.91      0.62      0.74        16
             american football       0.00      0.00      0.00        10
                 art and music       0.00      0.00      0.00        13
                      baseball       0.00      0.00      0.00         5
                    basketball       0.00      0.00      0.00        14
                        beauty       0.00      0.00      0.00        40
                     beverages       0.00      0.00      0.00        19
                  bodyb

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# Define the neural network model
vgg1 = Sequential([
    Dense(y.shape[1], activation='sigmoid') 
])

# Compile the model
vgg1.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
vgg1.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2,validation_data=(X_test, y_test), callbacks=[F1ScoreCallback()])

Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step   accuracy: 0.0426 - loss: 0.
 - val_f1_score: 0.2270
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.0431 - loss: 0.4442 - val_accuracy: 0.1586 - val_loss: 0.1403
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2404 - loss: 0.122
 - val_f1_score: 0.3518
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2419 - loss: 0.1217 - val_accuracy: 0.3289 - val_loss: 0.1011
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3714 - loss: 0.090
 - val_f1_score: 0.4347
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3718 - loss: 0.0903 - val_accuracy: 0.3890 - val_loss: 0.0860
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4489 - loss: 0.078
 - val_f1_score: 0.4418
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4508 - loss: 0.0784 - val_accuracy: 0.4357 - val_loss: 0.0802
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5129 - loss: 0.071
 - val_f1_score: 0.4728
75/75 ━

In [26]:
# Evaluate the model
y_pred = vgg1.predict(X_test)
print(classification_report(y_test, y_pred > 0.5, target_names=mlb.classes_))

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
                                precision    recall  f1-score   support

                   Auto racing       1.00      1.00      1.00        21
                      Business       0.81      0.70      0.75        50
                         Games       1.00      0.57      0.73        14
                   Health care       1.00      1.00      1.00        16
             Physical exercise       0.92      0.94      0.93        35
            alcoholic beverage       1.00      1.00      1.00        16
             american football       1.00      1.00      1.00        10
                 art and music       1.00      1.00      1.00        13
                      baseball       1.00      1.00      1.00         5
                    basketball       1.00      0.93      0.96        14
                        beauty       1.00      1.00      1.00        40
                     beverages       0.83      1.00      0.90        19
                  bodyb

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
